In [3]:
import networkx as nx
import urllib.request
import json
import pandas as pd
import re
import pickle
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

### Now that we cleaned up the data we can build the network.

In [4]:
data = pd.read_pickle('LargeData/edges.pkl')
author_posts = dict()
for post in tqdm(range(data.shape[0])):
    # Create a dictionary of authors and their flair, who they answered and their comment
    author_name = data['author'][post]
    author_flair = data['flair'][post]
    answered = data['addressee'][post]
    comment = data['comment'][post]
    comment_id = data['id'][post]
    
    # Add author to dictionary or add attributes
    if author_name not in author_posts:
        author_posts[author_name] = [author_flair,[answered],[comment],[comment_id]]

    else:
        author_posts[author_name][1].append(answered)
        author_posts[author_name][2].append(comment)
        author_posts[author_name][3].append(comment_id)

100%|██████████| 1245214/1245214 [00:19<00:00, 63036.56it/s]


In [5]:
data_path = 'data/'
data_path = 'data/'
file = 'links2022.ndjson'

data = pd.read_json(data_path + file, lines=True)
data = data.rename(columns={'out_going':'author','in_going':'addressee'})
author_posts = dict()
for post in tqdm(range(data.shape[0])):
    # Create a dictionary of authors and their flair, who they answered and their comment
    author_name = data['author'][post]
    author_flair = data['flair'][post]
    answered = data['addressee'][post]
    comment = data['comment'][post]
    comment_id = data['id'][post]
    
    # Add author to dictionary or add attributes
    if author_name not in author_posts:
        author_posts[author_name] = [author_flair,[answered],comment,[comment_id]]
    else:
        author_posts[author_name][1].append(answered)
        author_posts[author_name][2] += comment
        author_posts[author_name][3].append(comment_id)

100%|██████████| 1245214/1245214 [00:35<00:00, 35007.66it/s]


### Build Graph

In [6]:
# Create a directed graph
DirectedGraph = nx.MultiDiGraph()
for author_name in author_posts.keys():
    # Add nodes to the graph
    DirectedGraph.add_node(author_name,flair=author_posts[author_name][0],comments=author_posts[author_name][2])

for author_name in author_posts.keys():
    # Add edges
    for edge_to in author_posts[author_name][1]:
        if edge_to != None:
            DirectedGraph.add_edge(author_name,edge_to,comments=author_posts[author_name][2],comment_id=author_posts[author_name][3])

# Create folder if it does not exist
if not os.path.exists('LargeData'):
    os.makedirs('LargeData')
    
# Save authors as a pickle file
pickle.dump(DirectedGraph, open('LargeData/Graph.pkl','wb'))

### Save data

In [7]:
pickle.dump(data, open('LargeData/data2022.pkl','wb'))